In [ ]:
using DICOM
using Juliana
# using Plots

In [ ]:
output_dir = "tmp_dicom_output"
mkpath(output_dir);

# CT

In [ ]:
ct_0 = DICOM.dcm_parse("/data/user/bellotti_r/data/DICOM/train_00/CT.0.dcm");

In [ ]:
ct = Juliana.load_ct_dat_file("/data/user/bellotti_r/data/CTs/train_00_0.dat");

In [ ]:
dicom_dir = "/data/user/bellotti_r/data/DICOM/train_00"
ct_files = [f for f in readdir(dicom_dir) if startswith(f, "CT.")]
ct_files = sort(ct_files, by=text -> parse(Int64, split(text, ".")[2]));
ct_paths = ["$(dicom_dir)/$f" for f in ct_files];

In [ ]:
ct2 = Juliana.read_dicom_ct(ct_paths);

In [ ]:
@assert ct.grid.origin == ct2.grid.origin
@assert ct.grid.spacing == ct2.grid.spacing
@assert ct.grid.size == ct2.grid.size
@assert ct.data == ct2.data

In [ ]:
slice_ind = 0
# Transpose: DICOM is row-major, Julia is column-major.
# https://dicom.innolitics.com/ciods/ct-image/image-pixel/7fe00010
array = ct_0.PixelData' .* ct_0.RescaleSlope .+ ct_0.RescaleIntercept
spacing = convert.(Float32, round.([
    ct_0.PixelSpacing[1] / 10,
    ct_0.PixelSpacing[2] / 10,
    ct_0.SliceThickness  / 10,
], digits=6)) # cm
origin = ct_0.ImagePositionPatient ./ 10 # cm
orientation = ct_0.PatientPosition
decrease_precision = true

dicom_ct = Juliana.ct_slice_to_dicom(array, spacing, origin, orientation, slice_ind, decrease_precision=true);

In [ ]:
DICOM.dcm_write("CT.test.0.dcm", dicom_ct);
loaded = DICOM.dcm_parse("CT.test.0.dcm");

In [ ]:
@assert loaded.PixelData == ct_0.PixelData
@assert loaded.RescaleIntercept == ct_0.RescaleIntercept
@assert loaded.RescaleSlope == ct_0.RescaleSlope
@assert loaded.Columns == ct_0.Columns
@assert loaded.Rows == ct_0.Rows
@assert loaded.ImageOrientationPatient == ct_0.ImageOrientationPatient
@assert loaded.ImagePositionPatient == ct_0.ImagePositionPatient
@assert loaded.NumberOfFrames == ct_0.NumberOfFrames
@assert loaded.PixelSpacing == ct_0.PixelSpacing
@assert loaded.SliceLocation == ct_0.SliceLocation
@assert loaded.SliceThickness == ct_0.SliceThickness

In [ ]:
patient_ID = "train_00"
new_patient_ID = "bellotti_r_dicom_test"
study_instance_UID = Juliana.get_study_instance_uid(new_patient_ID)

frame_of_reference_UID = "$(study_instance_UID).0"
ct_series_instance_UID = "$(study_instance_UID).1"
structureset_series_instance_UID = "$(study_instance_UID).2"
patient_name = "$(new_patient_ID)^$(new_patient_ID)"

output_dir = "dicom_test"
mkpath(output_dir)

ct_datasets = Juliana.ct_to_dicom(
    ct,
    study_instance_UID,
    frame_of_reference_UID,
    ct_series_instance_UID,
    new_patient_ID,
    patient_name,
)
for ds in ct_datasets
    ds.SeriesDescription = "CT Images"
    dcm_write("$(output_dir)/CT.$(ds.InstanceNumber).dcm", ds)
end

In [ ]:
paths = ["$(output_dir)/CT.$(i).dcm" for i in 0:length(ct_datasets)-1]
ct_loaded = Juliana.read_dicom_ct(paths);

In [ ]:
@assert ct.data == ct_loaded.data
@assert ct.grid.origin == ct_loaded.grid.origin
@assert ct.grid.spacing == ct_loaded.grid.spacing
@assert ct.grid.size == ct_loaded.grid.size

# Dose

In [ ]:
dose_orig = Juliana.read_dicom_dose("/data/user/bellotti_r/data/DICOM/train_00/RD_plan_psiplan.dcm");

In [ ]:
i = 0
series_instance_UID = "$(study_instance_UID).$(3+i)"

In [ ]:
dose_dat = Juliana.load_dose_dat_file("/data/user/bellotti_r/data/clinical_dose_distributions/train_00_0.dat");

In [ ]:
data = dose_orig.data
spacing = dose_orig.grid.spacing
origin = dose_orig.grid.origin

dicom_dose = Juliana.dose_to_dicom(
    dose_orig,
    patient_name,
    new_patient_ID,
    study_instance_UID,
    series_instance_UID,
    frame_of_reference_UID,
)
DICOM.dcm_write("RT_dose.test.dcm", dicom_dose);

In [ ]:
loaded = Juliana.read_dicom_dose("RT_dose.test.dcm");

In [ ]:
@assert maximum(abs.((dose_dat.data .- dose_orig.data))) < 5e-3

In [ ]:
@assert dose_orig.grid.origin == loaded.grid.origin
@assert dose_orig.grid.spacing == loaded.grid.spacing
@assert dose_orig.grid.size == loaded.grid.size
@assert maximum(abs.(loaded.data .- dose_orig.data)) < 1e-3

# Structure Set

## Reading

In [ ]:
structure_points = Juliana.read_dicom_structure_points("/data/user/bellotti_r/data/DICOM/train_00/RS.1.2.826.0.1.3680043.8.498.98406890352034867029780269302956614897.2.dcm");

In [ ]:
structures = Juliana.read_dicom_structureset(
    "/data/user/bellotti_r/data/DICOM/train_00/RS.1.2.826.0.1.3680043.8.498.98406890352034867029780269302956614897.2.dcm",
    ct.grid,
);

In [ ]:
for name in keys(structure_points)
    @assert structures[name].points == structure_points[name]
end
@assert keys(structure_points) == keys(structures)

In [ ]:
for name in collect(keys(structure_points))[1:3]
    points_old = Juliana.load_npy_structure(
        name,
        "/data/user/bellotti_r/data/structures/train_00/0/$(name).npy",
        ct.grid,
        false,
    ).points;
    points_old == structure_points[name]
end

## Writing

In [ ]:
structures_vect = collect(values(structures));

In [ ]:
dicom_structures = Juliana.structures_to_dicom(
    structures_vect,
    study_instance_UID,
    frame_of_reference_UID,
    ct_series_instance_UID,
    structureset_series_instance_UID,
    patient_ID,
    patient_name,
    ct,
    ct_datasets,
    drop_precision=true,
)
DICOM.dcm_write("RS.test.dcm", dicom_structures);

In [ ]:
structures_loaded = Juliana.read_dicom_structureset("RS.test.dcm", ct.grid);

In [ ]:
@assert Set(keys(structures)) == Set(keys(structures_loaded))

for name in keys(structures)
    @assert maximum(abs.(structures[name].points - structures_loaded[name].points)) < 1e-6
    # Masks are not calculated for performance reasons.
    # But if the points are the same, so will be the masks.
    # @assert structures[name].mask == structures_loaded[name].mask
end

# Convenience function for exporting everything

In [ ]:
doses = Dict{String, Juliana.ScalarGrid}()
doses["my_dose"] = dose_dat;

In [ ]:
new_patient_ID = "test_juliana_dicom_train_00"
study_instance_UID = Juliana.get_study_instance_uid(new_patient_ID);

In [ ]:
Juliana.dicom_export_to_directory(
    ct,
    structures,
    output_dir,
    study_instance_UID,
    new_patient_ID,
    doses,
)